# `almop` Library Setup

I'm developping a python library for helping me encapsulate analysis of causal structures in time series data. 
The library setup is as follows:
1. Initialize the logger and specify the logging level
2. Import local `almop` package path so that python will be able to call all needed functions
3. Import `almop` modules and all other libraries

## Step 1 - Logging

In [1]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format='[%(levelname)s] %(asctime)s:\t %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Step 2 - System Path

In [2]:
import sys
import os

if f"{os.getcwd()}/../" not in ' '.join(sys.path):
    logging.info('Local module \'almop\' not found in system path. Appending...')
    sys.path.append(f"{os.getcwd()}/../")

[INFO] 2023-08-19 18:29:50,401:	 Local module 'almop' not found in system path. Appending...


## Step 3 - Import Libraries

In [3]:
#from almop import data as ald
from almop import data as ald
from almop import core as alc
from almop import gpu as alg
from almop import math as alm
import pandas as pd

## Parse Config

I'm using a configuration file that all notebooks can refer to. It contains useful parameters defining execution, such as the location on disk where the datasets should be persisted to.

**NOTE**: If you're running this for yourself, make sure to modify the config file appropriately. Everything else should then work smoothly.

In [4]:
config = alc.parse_config('config.toml')
config

{'dataset_persist_dir': '/wd4own/'}

## Loading Data

In [5]:
gdp = ald.get_gdp(config, ['PL', 'DE', 'IT', 'BE', 'SE'])

[INFO] 2023-08-19 18:29:51,558:	 Successfully loaded 'eurostat/NAMQ_10_GDP' from disk.


## Setting up `CuPy`

I will use the `gpu` part of the `almop` package to initialize interactions with the system's GPUs. The goal of the setup is to prepare the environment for executing CPU/GPU agnostic code.

In [6]:
import cupy as cp 

alg.set_memory_limits() # Set GPU V-RAM limits for all units

[INFO] 2023-08-19 18:29:51,680:	 GPU module found 2 physical devices. Setting memory limits...


In [11]:
a = gdp['PL'].dropna().values

<module 'numpy' from '/usr/local/lib/python3.8/dist-packages/numpy/__init__.py'>

In [12]:
a = cp.asarray(a)
gpu_result = alm.zscore(a)
cpu_result = cp.asnumpy(gpu_result)
del gpu_result

In [13]:
cpu_result

array([-1.51882359, -1.50497322, -1.46687385, -1.49540032, -1.40330453,
       -1.38082068, -1.37651071, -1.36204037, -1.30165208, -1.27959327,
       -1.27834522, -1.2811689 , -1.20585004, -1.16927216, -1.18514486,
       -1.19671843, -1.21918603, -1.17272393, -1.12745031, -1.11635051,
       -1.02514812, -1.0041668 , -0.94494264, -0.90524599, -0.8530553 ,
       -0.71607844, -0.81608758, -0.76984746, -0.72202901, -0.7629304 ,
       -0.87813   , -0.84548578, -0.9014152 , -0.92286488, -0.92810615,
       -0.96298928, -0.9533108 , -0.90730892, -0.79997663, -0.68515604,
       -0.60521317, -0.61416068, -0.54784348, -0.49874991, -0.41630011,
       -0.43408685, -0.37929989, -0.27911479, -0.24535788, -0.14609866,
       -0.09725415,  0.0524631 ,  0.15128646,  0.30230591,  0.41110554,
        0.11955729, -0.2063597 , -0.17397267, -0.00982298,  0.0277972 ,
        0.12623341,  0.16566204,  0.20753533,  0.2713808 ,  0.35857915,
        0.38827245,  0.31247711,  0.20880775,  0.32552072,  0.33